# Part A: Prepare a 🤗 Hugging Face Dataset from Data from CSV Format

In [ ]:
!pip install datasets transformers shutup ipython-autotime evaluate shutup --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
dataset_file = 'dataset-filtered.csv'
model_dipesh = 'Intent-Classification-small'
tokenizer_dipesh = 'Intent-Classification-small'
dataset_dipesh = 'Intent-Classification-small'
per_device_train_batch_size=64
per_device_eval_batch_size=64
num_train_epochs=8
base_model = 'joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english'

time: 1.05 ms (started: 2023-01-25 21:23:52 +00:00)


In [ ]:
%load_ext autotime
import shutup
shutup.please()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.67 ms (started: 2023-01-25 21:18:51 +00:00)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

pd.set_option('max_colwidth', 400)

time: 554 µs (started: 2023-01-25 21:18:51 +00:00)


In [ ]:
base_path = '/content/drive/MyDrive/Machine_Learning/Transformers-Classification/'

time: 486 µs (started: 2023-01-25 21:18:52 +00:00)


In [ ]:
dataset = load_dataset("csv", data_files=base_path+dataset_file)

  0%|          | 0/1 [00:00<?, ?it/s]

time: 954 ms (started: 2023-01-25 21:18:52 +00:00)


In [ ]:
dataset = dataset["train"].shuffle(seed=42)

time: 6.36 ms (started: 2023-01-25 21:18:53 +00:00)


In [ ]:
dataset = dataset.rename_column(
    original_column_name="labels", new_column_name="label"
)
# dataset = dataset.remove_columns('labels')

time: 2.2 ms (started: 2023-01-25 21:18:53 +00:00)


In [ ]:
dataset = dataset.train_test_split(train_size=0.9, seed=42)
# Rename the default "test" split to "validation"
dataset["validation"] = dataset.pop("test")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 684
    })
})

time: 9.58 ms (started: 2023-01-25 21:18:53 +00:00)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english')

dataset = dataset.map(lambda examples: tokenizer(examples["text"], return_tensors="np"), batched=True)

time: 1 s (started: 2023-01-25 21:18:53 +00:00)


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
})

time: 5.91 ms (started: 2023-01-25 21:18:54 +00:00)


### Generating id2label, label2id and num_labels

In [ ]:
import json
import pandas as pd

df = pd.read_csv(base_path+dataset_file)
dict_labels = {}
for index, row in df.iterrows():
    if row['labels'] not in dict_labels:
        dict_labels[row['labels']] = row['intent']

# to json
with open('labels.json', 'w') as f:
    json.dump(dict_labels, f)

# load json file
with open('labels.json', 'r') as f:
    dict_labels = json.load(f)


id2label = dict_labels
label2id = {v: int(k) for k, v in dict_labels.items()}
num_labels = len(df.labels.unique())
num_lael = dict_labels
label2id = {v: int(k) for k, v in dict_labels.items()}
num_labels = len(df.labels.unique())
num_labels

21

time: 255 ms (started: 2023-01-25 21:18:54 +00:00)


In [ ]:
from datasets import Dataset, Value, ClassLabel, Features

# Creating a ClassLabel Object
df = dataset["train"].to_pandas()
labels = df['intent'].unique().tolist()
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

dataset = dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
})

time: 167 ms (started: 2023-01-25 21:18:55 +00:00)


In [ ]:
dataset = dataset.align_labels_with_mapping(label2id, "label")

Aligning the labels:   0%|          | 0/7 [00:00<?, ?ba/s]

Aligning the labels:   0%|          | 0/1 [00:00<?, ?ba/s]

time: 612 ms (started: 2023-01-25 21:18:55 +00:00)


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
})

time: 3.12 ms (started: 2023-01-25 21:18:56 +00:00)


In [ ]:
dataset.push_to_hub(f"dipesh/{dataset_dipesh}")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

time: 21.8 s (started: 2023-01-25 21:18:56 +00:00)


# PART B: Hugging Face Tokenization: Use a Pre-trained Tokenizer or Train a New Tokenizer from scratch?

As suggested in the official documentation, if a language model is not available in the language you are interested in, or if your corpus is very different from the one your language model was trained on, you will most likely want to retrain the model from scratch using a tokenizer adapted to your data. That will require training a new tokenizer on your dataset.

## Using Pretrained

In [ ]:
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)

outputs = pretrained_tokenizer(
    dataset["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print("vocab_size: ", len(pretrained_tokenizer))

Input IDs length: 2
Input chunk lengths: [6, 9]
vocab_size:  30522
time: 928 ms (started: 2023-01-25 21:19:18 +00:00)


In [ ]:
txt = "how are you"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)

[101, 2129, 2024, 2017, 102]
time: 982 µs (started: 2023-01-25 21:19:18 +00:00)


In [ ]:
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 609 µs (started: 2023-01-25 21:19:19 +00:00)


## Train a new tokenizer

In [ ]:
def get_training_corpus():
    batch_size = 1000
    return (
        dataset["train"][i : i + batch_size]["text"]
        for i in range(0, len(dataset["train"]), batch_size)
    )
training_corpus = get_training_corpus()

time: 1.61 ms (started: 2023-01-25 21:19:19 +00:00)


In [ ]:
for data in get_training_corpus():
    print(len(data))

1000
1000
1000
1000
1000
1000
153
time: 118 ms (started: 2023-01-25 21:19:19 +00:00)


### NOTE: Now that we have our corpus in the form of an iterator of batches of texts, we are ready to train a new tokenizer. To do this, we first need to load the tokenizer we want to pair with our model

### NOTE: Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as facebook/bart-large-mnli, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.

In [ ]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)
vocab_size = len(pretrained_tokenizer)
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

time: 1.23 s (started: 2023-01-25 21:19:19 +00:00)


In [ ]:
print(tokenizer.eos_token_id, ",", tokenizer.vocab_size)

None , 3426
time: 2.2 ms (started: 2023-01-25 21:19:20 +00:00)


In [ ]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print(tokens)

[2, 180, 145, 88, 3]
time: 2.57 ms (started: 2023-01-25 21:19:20 +00:00)


In [ ]:
converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 413 µs (started: 2023-01-25 21:19:20 +00:00)


In [ ]:
print("pretrained_tokenizer-")
print(pretrained_tokenizer)

print("New tokenizer-")
print(tokenizer)

pretrained_tokenizer-
DistilBertTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
New tokenizer-
DistilBertTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=3426, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
time: 819 µs (started: 2023-01-25 21:19:20 +00:00)


In [ ]:
print("New tokenizer: ", len(tokenizer.tokenize(txt)))
print("Pretrained tokenizer: ", len(pretrained_tokenizer.tokenize(txt)))

New tokenizer:  3
Pretrained tokenizer:  3
time: 1.23 ms (started: 2023-01-25 21:19:20 +00:00)


### Saving tokenizer in local

In [ ]:
path=f"/"
file_name=model_dipesh
tokenizer.save_pretrained(path+file_name)

('/Intent-Classification-small/tokenizer_config.json',
 '/Intent-Classification-small/special_tokens_map.json',
 '/Intent-Classification-small/vocab.txt',
 '/Intent-Classification-small/added_tokens.json',
 '/Intent-Classification-small/tokenizer.json')

time: 19 ms (started: 2023-01-25 21:21:24 +00:00)


### Loading tokenizer from local

In [ ]:
loaded_tokenizer = AutoTokenizer.from_pretrained(path+file_name)

time: 4.4 ms (started: 2023-01-25 21:21:30 +00:00)


In [ ]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)

trained tokenizer: [2, 180, 145, 88, 3]
loaded tokenizer: [2, 180, 145, 88, 3]
time: 2 ms (started: 2023-01-25 21:21:32 +00:00)


In [ ]:
tokenizer.push_to_hub(f"dipesh/{tokenizer_dipesh}")

CommitInfo(commit_url='https://huggingface.co/dipesh/Intent-Classification-small/commit/d7368e5ae8c451cc688bac2bf45c7cddffcdbcab', commit_message='Upload tokenizer', commit_description='', oid='d7368e5ae8c451cc688bac2bf45c7cddffcdbcab', pr_url=None, pr_revision=None, pr_num=None)

time: 3.78 s (started: 2023-01-25 21:21:43 +00:00)


You can then load the tokenizer from anywhere with the from_pretrained() method:

In [ ]:
downloaded_tokenizer = AutoTokenizer.from_pretrained(f"dipesh/{model_dipesh}")

txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained (New) tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)
tokens = downloaded_tokenizer(txt)['input_ids']
print("downloaded tokenizer:", tokens)

trained (New) tokenizer: [2, 180, 145, 88, 3]
loaded tokenizer: [2, 180, 145, 88, 3]
downloaded tokenizer: [2, 180, 145, 88, 3]
time: 4.59 s (started: 2023-01-25 21:21:51 +00:00)


# PART C: Training a model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizerFast, RobertaTokenizer
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification, \
    TFRobertaForSequenceClassification, TFAutoModel, AutoModel, AutoModelForSequenceClassification
from sklearn import preprocessing
from datasets import load_dataset

time: 2.38 s (started: 2023-01-25 21:22:17 +00:00)


### Loading Dataset

In [ ]:
dataset = load_dataset(f'dipesh/{dataset_dipesh}')
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/684 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/6153 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/dipesh___parquet/dipesh--Intent-Classification-small-d9d04895d8cd2956/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
    train: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
})

time: 12.9 s (started: 2023-01-25 21:22:22 +00:00)


### Creating Datacollector and evaluator

In [ ]:
from transformers import DataCollatorWithPadding
import evaluate


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

time: 3.55 s (started: 2023-01-25 21:22:41 +00:00)


In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

time: 936 µs (started: 2023-01-25 21:23:28 +00:00)


### Loading Model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# If not loaded then load tokenizer by uncommenting below line
# tokenizer = AutoTokenizer.from_pretrained("dipesh/Intent-Classification-Commands-balanced-ds-large")
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=int(num_labels), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

### Creating tokenized dataset

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

time: 479 ms (started: 2023-01-25 21:25:53 +00:00)


In [ ]:
tokenized_dataset

DatasetDict({
    validation: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
    train: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
})

time: 3.21 ms (started: 2023-01-25 21:25:57 +00:00)


### Finetuning Model

In [ ]:
config = {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": id2label,
  "initializer_range": 0.02,
  "label2id": label2id,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": False,
  "tie_weights_": True,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

time: 1.54 ms (started: 2023-01-25 21:26:03 +00:00)


In [ ]:
with open(path+file_name+'/config.json', "w") as f:
    json.dump(config, f)

time: 1.07 ms (started: 2023-01-25 21:26:07 +00:00)


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(base_model)
config.update({"id2label": id2label})
config.update({"label2id": label2id})
# Push the config to your namespace with the name "my-finetuned-bert".
config.push_to_hub(f"dipesh/{model_dipesh}")

CommitInfo(commit_url='https://huggingface.co/dipesh/Intent-Classification-small/commit/6b58f4e241fb54610277218802dcd62dcf2cc516', commit_message='Upload config', commit_description='', oid='6b58f4e241fb54610277218802dcd62dcf2cc516', pr_url=None, pr_revision=None, pr_num=None)

time: 4.33 s (started: 2023-01-25 21:26:33 +00:00)


In [ ]:
import os
os.system('git init /content/Intent-Classification-small')

0

time: 48.8 ms (started: 2023-01-25 21:28:00 +00:00)


In [ ]:
output_dir_model_hf_name=f"dipesh/{model_dipesh}"

training_args = TrainingArguments(
    output_dir=output_dir_model_hf_name,
    learning_rate=2e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/dipesh/Intent-Classification-small into local empty directory.
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intent, text. If intent, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6153
  Num Epochs = 8
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 776
  Number of trainable parameters = 66969621
You're using a DistilBertTokeniz

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.196224,0.356725
2,No log,1.447817,0.616959
3,No log,0.958587,0.723684
4,No log,0.684590,0.858187
5,No log,0.528122,0.883041
6,1.458100,0.381132,0.916667
7,1.458100,0.352744,0.925439
8,1.458100,0.338751,0.923977


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intent, text. If intent, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 684
  Batch size = 64
Saving model checkpoint to dipesh/Intent-Classification-small/checkpoint-97
Configuration saved in dipesh/Intent-Classification-small/checkpoint-97/config.json
Model weights saved in dipesh/Intent-Classification-small/checkpoint-97/pytorch_model.bin
tokenizer config file saved in dipesh/Intent-Classification-small/checkpoint-97/tokenizer_config.json
Special tokens file saved in dipesh/Intent-Classification-small/checkpoint-97/special_tokens_map.json
tokenizer config file saved in dipesh/Intent-Classification-small/tokenizer_config.json
Special tokens file saved in dipesh/Intent-Classification-small/special_tokens_map.json
The following 

TrainOutput(global_step=776, training_loss=1.0861214214993506, metrics={'train_runtime': 164.8596, 'train_samples_per_second': 298.581, 'train_steps_per_second': 4.707, 'total_flos': 203182482107970.0, 'train_loss': 1.0861214214993506, 'epoch': 8.0})

time: 2min 54s (started: 2023-01-25 21:31:59 +00:00)


# Inference

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=f"./{output_dir_model_hf_name}")

loading configuration file ./dipesh/Intent-Classification-small/config.json
Model config DistilBertConfig {
  "_name_or_path": "./dipesh/Intent-Classification-small",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "asking date",
    "1": "asking time",
    "2": "asking weather",
    "3": "click photo",
    "4": "covid cases",
    "5": "download youtube video",
    "6": "goodbye",
    "7": "greet and hello hi kind of things, general check in",
    "8": "i am bored",
    "9": "open website",
    "10": "places near me",
    "11": "play games",
    "12": "play on youtube",
    "13": "send email",
    "14": "send whatsapp message",
    "15": "take screenshot",
    "16": "tell me about",
    "17": "tell me joke",
    "18": "tell me news",
    "19": "volume control",
    "20": "what can you do"
  },
  "initializer_range": 0.02,
  "label2id": {

time: 1.38 s (started: 2023-01-25 21:34:53 +00:00)


In [ ]:
classifier("click photo")

[{'label': 'covid cases', 'score': 0.8173684477806091}]

time: 284 ms (started: 2023-01-25 21:34:54 +00:00)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(
    f"dipesh/{tokenizer_dipesh}")
model = AutoModelForSequenceClassification.from_pretrained(
    f"dipesh/{model_dipesh}")
classifier_return_all_scores = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True )
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file dipesh/Intent-Classification-small/config.json
Model config DistilBertConfig {
  "_name_or_path": "dipesh/Intent-Classification-small",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "asking date",
    "1": "asking time",
    "2": "asking weather",
    "3": "click photo",
    "4": "covid cases",
    "5": "download youtube video",
    "6": "goodbye",
    "7": "greet and hello hi kind of things, general check in",
    "8": "i am bored",
    "9": "open website",
    "10": "places near me",
    "11": "play games",
    "12": "play on youtube",
    "13": "send email",
    "14": "send whatsapp message",
    "15": "take screenshot",
    "16": "tell me about",
    "17

time: 836 ms (started: 2023-01-25 21:34:54 +00:00)


In [ ]:
sequence = "make me laugh"

# Classify the sequence
results = classifier(sequence)

# Print the results
print(results)

[{'label': 'i am bored', 'score': 0.9581655263900757}]
time: 44 ms (started: 2023-01-25 21:34:55 +00:00)


In [ ]:
sequence = "make me laugh"

# Classify the sequence
results = classifier_return_all_scores(sequence)

# Print the results
print(results)

[[{'label': 'asking date', 'score': 0.00355823477730155}, {'label': 'asking time', 'score': 0.0007413921994157135}, {'label': 'asking weather', 'score': 0.0016888449899852276}, {'label': 'click photo', 'score': 0.003323993179947138}, {'label': 'covid cases', 'score': 0.0010668343165889382}, {'label': 'download youtube video', 'score': 0.0018192316638305783}, {'label': 'goodbye', 'score': 0.0010699485428631306}, {'label': 'greet and hello hi kind of things, general check in', 'score': 0.002550737466663122}, {'label': 'i am bored', 'score': 0.9581655263900757}, {'label': 'open website', 'score': 0.0006169233820401132}, {'label': 'places near me', 'score': 0.001818606280721724}, {'label': 'play games', 'score': 0.0019676117226481438}, {'label': 'play on youtube', 'score': 0.004100628662854433}, {'label': 'send email', 'score': 0.00694119930267334}, {'label': 'send whatsapp message', 'score': 0.001111537916585803}, {'label': 'take screenshot', 'score': 0.001140785519964993}, {'label': 'tel

time: 57.6 ms (started: 2023-01-25 21:34:55 +00:00)
